**All Packages Used**

In [63]:
import pandas as pd
from scipy.spatial import cKDTree
import folium
import json
import requests

**Combining Based on Closest Location**

In [98]:
# Sample DataFrames
df1 = pd.DataFrame({
    'latitude': [40.7128, 34.0522, 37.7749],
    'longitude': [-74.0060, -118.2437, -122.4194],
    'data1': ['A', 'B', 'C']
})

df2 = pd.DataFrame({
    'latitude': [41.8781, 34.0522, 37.7749],
    'longitude': [-87.6298, -118.2437, -122.4194],
    'data2': [1, 2, 3]
})

# Create KDTree for df2
tree = cKDTree(df2[['latitude', 'longitude']])

# Function to find closest point in df2 for each row in df1
def find_closest(row):
    distance, index = tree.query([row['latitude'], row['longitude']])
    closest_data = df2.iloc[index]
    return pd.Series({'data2': closest_data['data2'], 'distance': distance})

# Apply the function to each row in df1
result = df1.apply(find_closest, axis=1)

# Combine the results with df1
combined_df = pd.concat([df1, result], axis=1)

print(combined_df)


   latitude  longitude data1  data2   distance
0   40.7128   -74.0060     A    1.0  13.673546
1   34.0522  -118.2437     B    2.0   0.000000
2   37.7749  -122.4194     C    3.0   0.000000


In [96]:
climbs = pd.read_csv('utah_climbs.csv')
weather = pd.read_csv('Utah_Weather_Stations.csv')
weather.head()

,Area Latitude,Area Longitude,OBJECTID,STATION_NA,STN_ID_,SECONDARY_,STN_AUTHOR,NETWORK_,LOCATION_O,TYPE_OF_OB,...,SOIL_TEMP,FUEL_TEMP,FUEL_MOIST,PRESSURE,SL_PRESSUR,ALTIMETER,WEATHER_CO,VISIBILITY,UVB,ELEVATION
0,-1.257919e+07,4.439105e+06,1,CANNAN,42115T04,,NWS,COOP,http://climate.usu.edu,ONCE DAILY,...,,,,,,,,,,5000.000000
1,-1.261042e+07,4.439189e+06,2,FORT PEARCE /ST GEORGE,FPWU1,DD09129A,NOAA/NWS,HADS,http://amazon.nws.noaa.gov/hads/charts/UT.html,HOURLY,...,,,,,,,,,,
2,-1.262424e+07,4.439830e+06,3,VIRGIN RIVER/ST GEORGE 10SW,VRSU1,,1668A424,HADS,http://amazon.nws.noaa.gov/hads/charts/UT.html,HOURLY,...,,,,,,,,,,
3,-1.252687e+07,4.440500e+06,4,KANAB MUNICIPAL AIRPORT,KKNB,99999,National Weather Service,NWS/FAA,http://www.faa.gov/asos/,HOURLY,...,,,,PR,SLP,A,WC,VIS,,4863.912000
4,-1.257535e+07,4.441154e+06,5,CW2921 Hildale,C2921,9999999,APRSWXNET/Citizen Weather Observer Program,APRSWXNET/CWOP,http://www.wxqa.com/states/UT.html,HOURLY,...,,,,PR,SLP,A,,,,5155.832000


In [101]:
# Create KDTree for df2
tree = cKDTree(weather[['Area Latitude', 'Area Longitude']])

# Function to find closest point in df2 for each row in df1
def find_closest(row):
    distance, index = tree.query([row['Area Latitude'], row['Area Longitude']])
    closest_data = weather.iloc[index]
    return pd.Series({'STATION_NA': closest_data['STATION_NA'], 'distance': distance})

# Apply the function to each row in df1
closest = climbs.apply(find_closest, axis=1)

# Combine the results with df1
merged = pd.concat([climbs, closest], axis=1)

merged

,Route,URL,Avg Stars,Rating,Pitches,Length,Area Latitude,Area Longitude,PG13,R,...,numVotes,numViews,Year,ViewsPerMonth,Shared_by,Month,Day,Date,STATION_NA,distance
0,Tiki Man,https://www.mountainproject.com/route/10571855...,4.0,5.13c,1,60.0,38.49691,-109.31021,False,False,...,11,9559,2004,42,Anonymous Coward,12,31,2004-12-31,CEDAR POINT,1.294648e+07
1,The Bleeding,https://www.mountainproject.com/route/10813817...,4.0,5.14b,1,60.0,38.49691,-109.31021,False,False,...,2,6238,2013,49,Orphaned,5,12,2013-05-12,CEDAR POINT,1.294648e+07
2,Doubloon,https://www.mountainproject.com/route/11953024...,3.0,5.14b,1,90.0,38.49691,-109.31021,False,False,...,1,953,2020,25,Ben Crawford,9,9,2020-09-09,CEDAR POINT,1.294648e+07
3,Prosthetics,https://www.mountainproject.com/route/11288012...,4.0,5.13d,1,90.0,38.49649,-109.31053,False,False,...,4,2868,2017,36,ben jammin,5,7,2017-05-07,CEDAR POINT,1.294648e+07
4,Donut Project,https://www.mountainproject.com/route/11607613...,3.0,5.14+,1,30.0,38.49649,-109.31053,False,False,...,1,1560,2018,26,j wharton,11,8,2018-11-08,CEDAR POINT,1.294648e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6102,Suavemente,https://www.mountainproject.com/route/12500311...,2.3,5.8,1,50.0,40.69886,-110.89815,False,False,...,3,51,2023,42,The Possum's Protégé,9,25,2023-09-25,CEDAR POINT,1.294649e+07
6103,The Indecisive,https://www.mountainproject.com/route/12508882...,2.0,5.8,1,50.0,40.69886,-110.89815,False,False,...,2,44,2023,44,Craig Martin,10,9,2023-10-09,CEDAR POINT,1.294649e+07
6104,Crack 6,https://www.mountainproject.com/route/12333204...,2.5,5.8+,1,40.0,40.59798,-109.73133,False,False,...,2,69,2022,6,Joe Kitz,10,27,2022-10-27,CEDAR POINT,1.294649e+07
6105,Pre-pubescent Pimpled Tortoises.,https://www.mountainproject.com/route/12063800...,3.0,5.7,1,30.0,40.89100,-109.76765,False,False,...,1,348,2021,11,Paully,5,1,2021-05-01,CEDAR POINT,1.294649e+07


**Creating Map**

In [3]:
# Sample DataFrame with latitude and longitude columns
data = {
    'latitude': [40.7128, 34.0522, 37.7749],
    'longitude': [-74.0060, -118.2437, -122.4194],
    'location': ['New York', 'Los Angeles', 'San Francisco']
}
df = pd.DataFrame(data)

# Create a folium map centered at a specific location (e.g., New York)
map_center = [40.7128, -74.0060]
my_map = folium.Map(location=map_center, zoom_start=4)

# Add markers to the map based on latitude and longitude coordinates from the DataFrame
for index, row in df.iterrows():
    folium.Marker([row['latitude'], row['longitude']], popup=row['location']).add_to(my_map)

# Save the map to an HTML file
my_map.save("map.html")

# You can also display the map directly in a Jupyter Notebook
# my_map


In [62]:
# Dictionary containing latitude and longitude for the specified states
states_data = {
    'New York': {'latitude': 40.7128, 'longitude': -74.0060},
    'California': {'latitude': 36.7783, 'longitude': -119.4179},
    'Texas': {'latitude': 31.9686, 'longitude': -99.9018},
    'Florida': {'latitude': 27.7663, 'longitude': -81.6868}
}

# {"type":"FeatureCollection","features":[
# {"type":"Feature","id":"AL","properties":{"name":"Alabama"},"geometry":{"type":"Polygon","coordinates":[[[-87.359296,35.0011

# Write the dictionary to a JSON file
with open('states_coordinates.json', 'w') as json_file:
    json.dump(states_data, json_file)

# Sample DataFrame with data for choropleth map
data = {
    'state': ['New York', 'California', 'Texas', 'Florida'],
    'value': [30, 20, 25, 15]
}
df = pd.DataFrame(data)

# Create a folium map centered at a specific location (e.g., United States)
map_center = [37.0902, -95.7129]
my_map = folium.Map(location=map_center, zoom_start=4)

# Create a choropleth map using the folium.Choropleth class
folium.Choropleth(
    geo_data='states_coordinates.json',  # GeoJSON file containing state boundaries
    name='choropleth',
    data=df,
    columns=['state', 'value'],  # DataFrame columns for state names and values
    key_on='name',  # Key in the GeoJSON file for state names
    fill_color='YlGn',  # Color scale for the choropleth map
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Value'  # Legend title
).add_to(my_map)

# Display the map
#my_map.save('choropleth_map.html')
my_map


KeyError: 'type'

In [91]:
state_geo = requests.get(
    "https://raw.githubusercontent.com/python-visualization/folium-example-data/main/us_states.json"
).json()
state_data = pd.read_csv(
    "https://raw.githubusercontent.com/python-visualization/folium-example-data/main/us_unemployment_oct_2012.csv"
)

t = folium.Map(location=[48, -102], zoom_start=3)

folium.Choropleth(
    geo_data=state_geo,
    name="choropleth",
    data=state_data,
    columns=["State", "Unemployment"],
    key_on="feature.id",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Unemployment Rate (%)",
).add_to(t)
folium.Marker(
    location=[37.8, -113.6625],
    tooltip="Weather!",
    popup="Mt. Hood Meadows",
    icon=folium.Icon(icon="cloud"),
).add_to(t)

folium.LayerControl().add_to(t)
t

In [92]:
climbs.head()

,Route,URL,Avg Stars,Rating,Pitches,Length,Area Latitude,Area Longitude,PG13,R,...,Mixed,Rating_num,numVotes,numViews,Year,ViewsPerMonth,Shared_by,Month,Day,Date
0,Tiki Man,https://www.mountainproject.com/route/10571855...,4.0,5.13c,1,60.0,38.49691,-109.31021,False,False,...,False,13.6,11,9559,2004,42,Anonymous Coward,12,31,2004-12-31
1,The Bleeding,https://www.mountainproject.com/route/10813817...,4.0,5.14b,1,60.0,38.49691,-109.31021,False,False,...,False,14.4,2,6238,2013,49,Orphaned,5,12,2013-05-12
2,Doubloon,https://www.mountainproject.com/route/11953024...,3.0,5.14b,1,90.0,38.49691,-109.31021,False,False,...,False,14.4,1,953,2020,25,Ben Crawford,9,9,2020-09-09
3,Prosthetics,https://www.mountainproject.com/route/11288012...,4.0,5.13d,1,90.0,38.49649,-109.31053,False,False,...,False,13.8,4,2868,2017,36,ben jammin,5,7,2017-05-07
4,Donut Project,https://www.mountainproject.com/route/11607613...,3.0,5.14+,1,30.0,38.49649,-109.31053,False,False,...,False,14.7,1,1560,2018,26,j wharton,11,8,2018-11-08


In [90]:
utah = folium.Map(location=(40, -111), zoom_start=6.5, tiles="cartodb positron")

folium.Marker(
    location=[37.8, -113.6625],
    tooltip="Weather!",
    popup="Mt. Hood Meadows",
    icon=folium.Icon(icon="cloud"),
).add_to(utah)
folium.Marker(
    location=[40.2, -110.7113],
    tooltip="Info!",
    popup="Timberline Lodge",
    icon=folium.Icon(color="green"),
).add_to(utah)
folium.Marker(
    location=[climbs['Area Latitude'][0], climbs['Area Longitude'][0]],
    tooltip=climbs['Route'][0],
    popup=climbs['Rating'][0],
    icon=folium.Icon(prefix='fa', color="red", icon='hill-rockslide'),
).add_to(utah)
folium.Marker(
    location=[climbs['Area Latitude'][12], climbs['Area Longitude'][12]],
    tooltip=climbs['Route'][12],
    popup=climbs['Rating'][12],
    icon=folium.Icon(prefix='fa', icon="hill-rockslide"),
).add_to(utah)
folium.Marker(
    location=[climbs['Area Latitude'][20], climbs['Area Longitude'][20]],
    tooltip=climbs['Route'][20],
    popup=climbs['Rating'][20],
    icon=folium.Icon(icon="thumbs-up"),
).add_to(utah)
folium.Marker(
    location=[climbs['Area Latitude'][300], climbs['Area Longitude'][300]],
    tooltip=climbs['Route'][300],
    popup=climbs['Rating'][300],
    icon=folium.Icon(icon="thumbs-down"),
).add_to(utah)
folium.Marker(
    location=[climbs['Area Latitude'][100], climbs['Area Longitude'][100]],
    tooltip=climbs['Route'][100],
    popup=climbs['Rating'][100],
    icon=folium.Icon(icon="star"),
).add_to(utah)
folium.Marker(
    location=[climbs['Area Latitude'][1000], climbs['Area Longitude'][1000]],
    tooltip=climbs['Route'][1000],
    popup=climbs['Rating'][1000],
    icon=folium.Icon(prefix='fa', icon="star"),
).add_to(utah)

utah

In [117]:
utah = folium.Map(location=(39.5, -111), zoom_start=6.5, tiles="cartodb positron")

for i in range(1, 200):
    folium.Marker(
        location=[climbs['Area Latitude'][i], climbs['Area Longitude'][i]],
        tooltip=climbs['Route'][i],
        popup=climbs['Rating'][i],
        icon=folium.Icon(prefix='fa', icon="star"),
    ).add_to(utah)

utah.save('routes_map.html')
utah